<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#methods" data-toc-modified-id="methods-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>methods</a></span></li></ul></li><li><span><a href="#Planning-new-network" data-toc-modified-id="Planning-new-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Planning new network</a></span></li><li><span><a href="#go-through-the-steps-to-make-sure-i-remember" data-toc-modified-id="go-through-the-steps-to-make-sure-i-remember-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>go through the steps to make sure i remember</a></span><ul class="toc-item"><li><span><a href="#re-organize-data" data-toc-modified-id="re-organize-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>re-organize data</a></span></li><li><span><a href="#get_pairs()" data-toc-modified-id="get_pairs()-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get_pairs()</a></span></li></ul></li></ul></div>

## imports

In [1]:
import keras
import numpy as np
import datetime

from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict
from glob import glob
from random import choice, sample

# Florencia's imports
import tensorflow
from tensorflow.keras import layers, preprocessing, models, optimizers
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D,Activation, Dropout, BatchNormalization,  GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Subtract,Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
import random
import cv2
import os
import pickle
from os import listdir
from tqdm import tqdm
from tensorflow.keras.applications.resnet50 import preprocess_input
from random import sample, choice

from tensorflow.keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.backend import clear_session
from random import randint
from random import uniform as rfloat
import h5py
from tensorflow.keras.models import model_from_json 
from keras import regularizers
import os 

Using TensorFlow backend.


## methods

In [7]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img)
def paths_to_arrays(paths_list):
    out = []
    for path in paths_list:
        out.append(read_img(path))
    return out

def get_jpg_lists(): # hardcoded ;(
    dir_path = os.getcwd()
    train_part = '/UTKface/train/'
    val_part = '/UTKface/validation/'
    test_part = '/UTKface/test/'
    # no overlap between train, val, test imgs
    train_path = dir_path+train_part # 2008
    val_path = dir_path+val_part # 95
    test_path = dir_path+test_part # 155, distinct from val
    
    trains = glob(train_path + '*.jpg')
    tests = glob(test_path + '*.jpg')
    vals = glob(val_path + '*.jpg')
    return trains, tests, vals

def jpgs_to_arrays(trains, tests, vals):
    train_data = paths_to_arrays(trains)
    test_data = paths_to_arrays(tests)
    val_data = paths_to_arrays(vals)
    return train_data, test_data, val_data

# Planning new network
- 2 limiting factors from last model:
    - data
        - 2,000 images is good, but i don't think augmentation will lead to great learning
        - use something like AT&Tfaces, which has ~40 'users' but 8 photos of each user
            - this is few-shot learning after all, the simulated extra shots may not be giving me the desired output
            - then again, it could also be:
    - architecture size
        - 6 layers, mixture of dense+conv+flattening. messed around with different activations
        - i think i should try changing this first, because it's something i want to learn for future models as well
- Model updates:
    - transfer learning
        - try importing VGGnet, Facenet etc and running
        - learn Sagemaker to run it so i'm not limited by local computing power


# go through the steps to make sure i remember

## re-organize data

## get_pairs()
core function is get_pairs().
the net intakes 2 images and outputs 1 or 0 for same or different label.
so our x is a list of [img_array_a, img_array_b] pairs, and our y is a list of [1, 0, 1] matching each pair

In [9]:
def get_pairs(imgs_list): # creates 50/50 split of 'same' and 'dif' image pairs & labels. len=n*5
    pairs = [] # x
    labels = [] # y
    print(f'making pairs from {len(imgs_list)} images...')
    for c, img in enumerate(imgs_list):
        temp = imgs_list # shallow copy messed this up
        for i in range(0,4): # make 4 'same' and 'dif' pairs for each base_image
            # same pair
            pairs.append([img, img])
            labels.append(1) # true
            # dif pair
            print(f'getting rand_ind between 0 and {len(temp)}')
            indx = randint(0,len(temp)-1)
            while indx==c: # if random index matches current iteration index
                indx = randint(0,len(temp)-1) # avoid deepcopy memory time
            print(f'index: {indx}')
            pairs.append([img, temp[indx]]) # make a "dif" pair of images
            labels.append(0)
            print(f'total {len(pairs)}')
        print(f'done with image #{c} \n')
    return pairs, labels # returns lists for x, y. pairs+labels matched up by index

- process
    - get all images in one big list
        - train_test_split into

In [ ]:
def xy_split(train_list, test_list, val_list):
    x_train, y_train = get_pairs(train_data)
    x_test, y_test = get_pairs(test_data)
    x_val, y_val = get_pairs(val_data)
    return x_train,y_train, x_test,y_test, x_val,y_val

In [8]:

train_data, test_data, val_data = get_jpg_lists()
train_data, test_data, val_data = jpgs_to_arrays(train_data, test_data, val_data)
print(np.shape(train_data))
print(np.shape(test_data)) # shape glitch. maybe a corrupted jpg?
print(np.shape(val_data))

(2008, 200, 200, 3)
(155,)
(95, 200, 200, 3)


In [ ]:
# fix the test jpg issue
def avoid_ghosts(train_jpgs, n=160): # 2008-160=1848. both fit batch_size=16
    tests = random.sample(train_jpgs, n)
    for jpg in tests:
        if jpg in train_jpgs:
            train_jpgs.remove(jpg)
    return tests
test_data = avoid_ghosts(train_data)